# Building a Tranformer Architecture

Attention mechanism: capture long-range dependencies. Use whole sequence to weight each element.
Create a vector describing the token's position in the sequence <- transformers process tokens simultaneously

In [3]:
import torch
import torch.nn as nn

#  Create a vector describing the token's position in the sequence
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_length=512):
        super(PositionalEncoder, self).__init__()
        
        # set max length and embedding size
        self.d_model = d_model
        self.max_seq_length = max_seq_length
        
        # create matrix for sequences up to max_seq_length
        pe = torch.zeros(max_seq_length, d_model)

        # position indices in the sequence
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        # scale position indices
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * -(math.log(10000.0) / d_model))
        # alternate sin and cos to assign unique position encodings to the matrix pe
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        # set pe as non-trainable
        self.register_buffer('pe', pe)

    def forward(self, x):
        """Add positional encodings pe to input tensor of sequence embeddings, X"""
        x = x + self.pe[:, :x.size(1)]
        return x

Self-attention: understand the interrelationship between words in a sequence.

1. Input: sequence of embeddings.
2. Project the sequence of embeddings into 3 sequences of same dimensions: Q:query, K:key, V:values, by applying 3 linear transformations, each with its own weights learned during training
3. Compute a Q-K similarity matrix of attention scores (scaled dot-product self-attention)
4. Apply softmax to get a matrix of attention weights

In practice, transformers paralellize attention heads

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        """
        num_heads: number of attention heads, each handling embeddings of size head_dim
        """
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        seld.d_model = d_model
        self.head_dim = d_model // num_heads

        # define 3 linear transformatins for attention input
        self.query_linear = nn.Linear(d_model, d_model)
        self.key_linear = nn.Linear(d_model, d_model)
        self.value_linear = nn.Linear(d_model, d_model)

        # define linear transformation for the final concatenated output
        self.output_linear = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        """Split the inputs accross attention heads"""
        x = x.view(batch_size, -1, self.num_heads, self.head_dim)
        return x.permute(0, 2, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.head_dim)

    def compute_attention(self, query, key, mask=None):
        """Computes attention weights inside each head."""
        scores = torch.matmul(query, key.permute(1, 2, 0))
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float("-1e9"))
        attention_weights = F.softmax(scores, dim=-1)
        return attention_weights

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # attention weights for Q, K, V
        query = self.split_heads(self.query_linear(query), batch_size)
        key = self.split_heads(self.key_linear(key), batch_size)
        value = self.split_heads(self.value_linear(value), batch_size)

        # concatenate
        attention_weights = self.compute_attention(query, key, mask)

        # project to the original model dimension
        output = torch.matmul(attention_weights, value)
        output = output.view(batch_size, self.num_heads, -1, self.head_dim).
                    permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)
        return self.output_linear(output)